In [1]:
import duckdb as ddb
import pandas as pd

In [2]:
con = ddb.connect("../air_quality.db")

con

In [3]:
df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()

df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,locationid,month,year,ingestion_datetime
0,225393,1303603,Damonsville-237689,2024-01-01 02:00:00,-25.625702,27.851825,so2,ppm,0.001821,225393,01,2024,2024-11-13 20:53:09.264
1,225393,1303603,Damonsville-237689,2024-01-01 08:00:00,-25.625702,27.851825,so2,ppm,0.005904,225393,01,2024,2024-11-13 20:53:09.264
2,225393,1303603,Damonsville-237689,2024-01-06 23:00:00,-25.625702,27.851825,so2,ppm,0.001914,225393,01,2024,2024-11-13 20:53:09.264
3,225393,1303603,Damonsville-237689,2024-01-07 00:00:00,-25.625702,27.851825,so2,ppm,0.001592,225393,01,2024,2024-11-13 20:53:09.264
4,225393,1303603,Damonsville-237689,2024-01-07 02:00:00,-25.625702,27.851825,so2,ppm,0.001307,225393,01,2024,2024-11-13 20:53:09.264


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111376 entries, 0 to 111375
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   location_id         111376 non-null  int64         
 1   sensors_id          111376 non-null  int64         
 2   location            111376 non-null  object        
 3   datetime            111376 non-null  datetime64[us]
 4   lat                 111376 non-null  float64       
 5   lon                 111376 non-null  float64       
 6   parameter           111376 non-null  object        
 7   units               111376 non-null  object        
 8   value               111376 non-null  float64       
 9   locationid          111376 non-null  int64         
 10  month               111376 non-null  object        
 11  year                111376 non-null  int64         
 12  ingestion_datetime  111376 non-null  datetime64[us]
dtypes: datetime64[us](2), float64

In [5]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,locationid,year,ingestion_datetime
count,1.113760e+05,1.113760e+05,111376,111376.000000,111376.000000,111376.000000,1.113760e+05,111376.0,111376
mean,4.333709e+05,2.432606e+06,2024-02-06 05:43:19.572618,-26.211440,28.124571,14.273528,4.333709e+05,2024.0,2024-11-13 21:17:12.713369
min,2.253930e+05,1.303367e+06,2023-12-31 23:00:00,-26.851461,27.289861,-999.000000,2.253930e+05,2024.0,2024-11-13 20:53:09.264000
25%,2.254310e+05,1.303512e+06,2024-01-20 02:00:00,-26.657017,27.867111,0.003164,2.254310e+05,2024.0,2024-11-13 21:05:51.141000
50%,2.254560e+05,1.303637e+06,2024-02-05 05:00:00,-26.252611,28.018873,5.492000,2.254560e+05,2024.0,2024-11-13 21:15:37.139000
75%,2.308210e+05,1.347155e+06,2024-02-24 13:00:00,-25.939300,28.236044,22.000000,2.308210e+05,2024.0,2024-11-13 21:25:35.497000
max,2.506086e+06,7.980517e+06,2024-03-31 22:00:00,-24.785269,29.112381,616.475000,2.506086e+06,2024.0,2024-11-14 00:21:09.352000
std,4.924581e+05,2.244512e+06,NaN,0.477592,0.415170,43.422324,4.924581e+05,0.0,NaN


In [6]:
df.describe(include='O')

,location,parameter,units,month
count,111376,111376,111376,111376
unique,63,3,2,3
top,North West University Vaal campus-241488,so2,µg/m³,01
freq,3657,45512,68310,50415


In [7]:
df[df.duplicated(subset=["location_id", "parameter", "units", "value", "datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,locationid,month,year,ingestion_datetime


In [8]:
df.groupby(by="parameter", as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,locationid,month,year,ingestion_datetime
0,pm10,35944,35944,35944,35944,35944,35944,35944,35944,35944,35944,35944,35944
1,pm25,29920,29920,29920,29920,29920,29920,29920,29920,29920,29920,29920,29920
2,so2,45512,45512,45512,45512,45512,45512,45512,45512,45512,45512,45512,45512


In [10]:
df[df["value"] < 0]["location"].unique()

array(['eMbalenhle North-237737', 'Lebohang-237755', 'Embalenhle-237759',
       'North West University Vaal campus-241488',
       'Eskom-Chicken Farm-243099', 'Eskom-Ezamokuhle-243099',
       'Eskom-Grootvlei-243102', 'Eskom-Randwater-243107',
       'Eskom-Chicken Farm-243112', 'Eskom-Ezamokuhle-243112',
       'Eskom-Kendal-243113', 'Eskom-Phola-243117', 'Tsakane-374670',
       'Mobile-Norplats-1417233', 'Smashblock-1417234',
       'AMSA Vanderbijlpark Works-1858312', 'Wonderkop Sibanye-1865667'],
      dtype=object)

In [ ]:
con.close()

: 